<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 2*

# Document Representations: Bag-Of-Words

In [1]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

     |████████████████████████████████| 1.5MB 3.8MB/s eta 0:00:01     |█████████████▌                  | 614kB 3.8MB/s eta 0:00:01
  Stored in directory: /home/nedderlander/.cache/pip/wheels/41/c8/31/48ace4468e236e0e8435f30d33e43df48594e4d53e367cf061
Successfully built nltk
  Found existing installation: nltk 3.4.3
    Uninstalling nltk-3.4.3:
      Successfully uninstalled nltk-3.4.3


[nltk_data] Downloading package punkt to
[nltk_data]     /home/nedderlander/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nedderlander/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

If you choose to not to scrape the data, there is a CSV with outdated data in the directory. Remeber, if you scrape Indeed, you're helping yourself find a job. ;)

In [2]:
##### Your Code Here #####
import requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd
import time

In [5]:
# pull a single page to test:

url = "https://www.indeed.com/jobs?q=data+scientist&l=Seattle&start=10"

# conduct a request
page = requests.get(url)

# specify format of page
soup = BeautifulSoup(page.text, "html.parser")

# pretty print
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/e7e2a1c/en_US.js" type="text/javascript">
  </script>
  <link href="/s/684a333/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist&amp;l=Seattle" rel="alternate" title="Data Scientist Jobs, Employment in Seattle, WA" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist&amp;l=Seattle" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=data+scientist&amp;l=Seattle" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
  </script>
  <m

In [13]:
def extract_job_title_from_result(soup):
    jobs = []
    for div in soup.find_all(name='div', attrs={'class':'title'}): #attrs={'class':'row'}
        for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
            jobs.append(a['title'])
    return(jobs)

extract_job_title_from_result(soup)

['Internal Audit - Forensic Data Scientist',
 'Data Scientist',
 'UW MEDICINE CALL CENTER REPRESENTATIVE',
 'Computer Scientist',
 'Data Scientist/Machine Learning Engineer',
 'Research Program Manager - Machine Learning',
 'Internship Opportunities for PhD Students: Data & Applied Sciences',
 'Data Scientist, Home Loans',
 'Data Scientist - AMZ3704',
 'Data & Applied Scientist',
 'Data Scientist, Zillow Offers',
 'AI Data Scientist',
 'Data Scientist',
 'Data Scientist II',
 'Data Scientist',
 'Behavioral Scientist',
 'Data Scientist',
 'Data Scient (Python)']

In [48]:
def extract_company_from_result(soup):
    companies = []
    for span in soup.find_all(name='span', attrs={'class':'company'}):
        for a in span.find_all(name='a', attrs={'data-tn-element' : 'companyName'}):
            companies.append(a.text.strip())
    return companies

extract_company_from_result(soup)

['PeopleTech',
 'eBay Inc.',
 'US Department of Agriculture - Milk Market Administrator',
 'ASSURANCE Independent Agents',
 'Blue Origin',
 'Microsoft',
 'Rover',
 'Microsoft',
 'T-Mobile',
 'Amazon.com Services, Inc.',
 'Beyondsoft Consulting',
 'Microsoft',
 'Seattle Cancer Care Alliance',
 'Weyerhaeuser']

In [49]:
def extract_summary_from_result(soup):
    summaries = []
    for div in soup.find_all(name='div', attrs={'class':'summary'}):
        summaries.append(div.text.strip())
    return summaries

extract_summary_from_result(soup)

['Job Summary Jr. Data scientist role Job description:  Job Type: Full time Job location: Redmond, WA We are seeking a Data Scientist with machine...',
 'Looking for a company that inspires passion, courage and imagination, where you can help shape the future of global commerce?...',
 'The candidate for Associate Scientist should have a PhD or MS in immunology, cell biology, chemistry or genetics....',
 'About You:. You are a qualified, self-motivated and talented scientist that desires to collaborate with a team of like-minded and hard-working scientists and...',
 'Position Title, Series, & Grade:. Dairy Scientist (Chemist), MA-1321; MA 11-12 (This is not equivalent to GS pay schedule). Vacancy Number:....',
 "Earn from the industry's best comp plan selling Medicare Supplement using ASSURANCE, and access our leading-edge insurance sales platform as an independent,...",
 'As part of a small, passionate and accomplished team of experts, you will lead efforts to identify, capture, develo

In [50]:
def extract_location_from_result(soup):
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)

extract_location_from_result(soup)

['Redmond, WA',
 'Seattle, WA',
 'Redmond, WA',
 'Seattle, WA',
 'Redmond, WA',
 'Bellevue, WA',
 'Seattle, WA',
 'Redmond, WA',
 'Redmond, WA']

In [106]:
max_results_per_city = 20

city_set = ['Seattle', 'New+York', 'Austen', 'San+Fransisco', 'LA']

columns = ['city', 'job_title', 'company_name', 'job_summary', 'location']

In [107]:
#scarping
for city in city_set:
    print(city)
    
    
    #work through results in groups of 10 and wait between requests
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' +str(city) 
                            + '&start='+str(start))
        time.sleep(1)
        
        #soup object
        soup = BeautifulSoup(page.text, "html.parser")
        
        for 
        #list of results
        job
        
        
        print(start,city)
        temp_city = [city]*10
        city_list.extend(temp_city)
        job_title.extend(extract_job_title_from_result(soup))
        company_name.extend(extract_company_from_result(soup))
        job_summary.extend(extract_summary_from_result(soup))
        location.extend(extract_location_from_result(soup))
        
#add to a dictionary
data = {
    'city' : [x for x in city_list],
    'job_title' : [job for job in job_title],
    'company_name': [name for name in company_name],
    'job_summary': [s for s in job_summary],
    'location' : [l for l in location]
}  

        

Seattle
0 Seattle
10 Seattle
New+York
0 New+York
10 New+York
Austen
0 Austen
10 Austen
San+Fransisco
0 San+Fransisco
10 San+Fransisco
LA
0 LA
10 LA


In [108]:
data

{'city': ['Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'Seattle',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'New+York',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'Austen',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  'San+Fransisco',
  '

In [110]:
df = pd.DataFrame(data)
df.head()

ValueError: arrays must all be same length

In [ ]:
df.shape

## 2) Use Spacy to tokenize / clean the listings 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

# 4) Visualize the most common word counts

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

 # 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.